## Data preparation

In [0]:
import pandas as pd
import numpy as np

In [48]:
data = pd.read_csv('NSDUH_2016_Tab.tsv', sep='\t')
data.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2506) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(56897, 2668)

All column names in the dataset make no sense, so great amount of time was spent on understanding the data using appr. 700-page documentation on every column(question in the survey) of 2668 columns.



In [49]:
data.head(5)

,QUESTID2,FILEDATE,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,POVERTY3,TOOLONG,TROUBUND,PDEN10,COUTYP4,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP
0,11635143,02/28/2018,1,99,99,16,9999,99,4,93,...,3.0,2,2,3,3,2,2,819.434247,40037,2
1,36845143,02/28/2018,1,99,99,15,9999,99,1,7,...,3.0,1,2,2,2,2,2,280.624352,40013,2
2,35755143,02/28/2018,1,99,99,26,9999,99,1,7,...,2.0,2,2,1,1,2,2,10133.833583,40036,2
3,94475143,02/28/2018,2,4,4,991,9991,91,91,91,...,3.0,2,2,1,1,2,2,2284.717175,40028,1
4,92675143,02/28/2018,1,99,99,5,9999,99,4,93,...,3.0,2,2,1,1,2,2,24815.892373,40009,2


#### Some values meaning(from documentation):

91 or 991 or 9991, etc. = NEVER USED [DRUG(s) OF INTEREST] 

93 or 993 or 9993, etc. = USED [DRUG] BUT NOT IN THE PERIOD OF INTEREST

94 or 994 or 9994, etc. = DON'T KNOW

97 or 997 or 9997, etc. = REFUSED

98 or 998 or 9998, etc. = BLANK (i.e., not answered; not asked the question)

99 or 999 or 9999, etc. = LEGITIMATE SKIP

94-7-8 - missing

81 (or 981, 9981, etc.) = NEVER USED [DRUG(s) OF INTEREST] Logicall yassigned

83 (or 983, 9983, etc.) = USED [DRUG] BUT NOT IN THE PERIOD OF INTEREST Logically assigned

85 (or 985, 9985, etc.) = BAD DATA Logically assigned (i.e., usually inconsistent with other data)

89 (or 989, or 9989, etc.) = LEGITIMATE SKIP Logically assigned

'IR' 'IMP REV' 'IMPUTATION REVISITED' - indicators for missing values except for IRSEX

"IMPUTATION INDICATOR" 'IMP IND' 'II' "II2"- Associated indicator variables tell which values are imputed and which ones are not

#### Columns cleaning
__note__ : I decided to use imputed values provided in the dataset, where possible. As documentation explains, they used special imputation techniques designed for this survey specifically and even for each SSregion specifically. 

To find useless columns and prefixes I investigated the documentation provided to this survey(NSDUH-2016-DS0001-info-codebook.pdf file in the repository). 

In [50]:
# drop columns with info about imputation
data.drop([col for col in data if col.startswith(('II', 'EI'))], axis=1, inplace=True)
data.shape

(56897, 2495)

In [51]:
# drop variables for which there exists imputed analogue
imputed_cols = [col for col in data if col.startswith('IR')]
data.drop([col.strip('IR') for col in imputed_cols], axis=1, inplace=True, errors='ignore')
data.shape

(56897, 2423)

In [52]:
useless_cols = ['QUESTID2', 'FILEDATE', 'ANALWT_C', 'VESTR', 'VEREP']
data.drop(useless_cols, axis=1, inplace=True, errors='ignore')
data.drop([col for col in imputed_cols if col.startswith('CATA')], axis=1, inplace=True, errors='ignore')
data.shape

(56897, 2418)

#### Missing value identification
__note__: again, from documentation. Values meanings listed above(in "Some values meaning" section).

In [0]:
data.replace([85, 985, 9985, 94, 994, 9994, 97, 997, 9997, 98, 998, 9998], np.nan, inplace=True)
data.to_csv('prepared_data.csv')

#### Target variable creation
I decided to study current users. For every type of drug (13 of them) I take question of the type "How long ago did you use [certain_drug]" and create a binary variable [certain_drug]_user. 1 - used a drug during last 12 monhs and 0 - otherwise.

In [54]:
recency_cols = [col for col in data if (col.endswith('REC') and not col.endswith('NDLREC'))]
recency_cols

['CIGREC',
 'CIGARREC',
 'ALCREC',
 'MJREC',
 'COCREC',
 'CRAKREC',
 'HERREC',
 'LSDREC',
 'PCPREC',
 'INHALREC',
 'IRSMKLSSREC',
 'IRHALLUCREC',
 'IRECSTMOREC',
 'IRKETMINREC',
 'IRDAMTFXREC',
 'IRSALVIAREC',
 'IRINHALREC',
 'IRMETHAMREC',
 'IRPNRANYREC',
 'IRTRQANYREC',
 'IRSTMANYREC',
 'IRSEDANYREC',
 'IRPNRNMREC',
 'IRTRQNMREC',
 'IRSTMNMREC',
 'IRSEDNMREC',
 'COLDREC',
 'GHBREC',
 'HRSMKREC',
 'HRSNFREC',
 'BLNTREC',
 'TXRCVDREC']

From upper output I select needed cols using docs:

In [55]:
pretarget_pairs = [('hal_user', 'IRHALLUCREC'),
                   ('inh_user', 'IRINHALREC'),
                   ('meth_user', 'IRMETHAMREC'), 
                   ('pnr_user', 'IRPNRNMREC'),
                   ('trq_user', 'IRTRQNMREC'),
                   ('stm_user', 'IRSTMNMREC'),
                   ('sed_user', 'IRSEDNMREC'),
                   ('cig_user', 'CIGREC'),
                   ('alc_user', 'ALCREC'),
                   ('mj_user', 'MJREC'),
                   ('coc_user', 'COCREC'),
                   ('crack_user', 'CRAKREC'),
                   ('her_user', 'HERREC')]

targets = pd.DataFrame()
for name,col in pretarget_pairs:
    targets[name] = data[col].replace([2,4,8], 1)
    targets[name].replace([3,4,9,11,14,19,29,39,91],0, inplace=True)
    
targets.to_csv('targets.csv')    
targets['alc_user'].unique()

array([ 1.,  0., nan])

mb for later - ''Have you ever tried'' type of questions:

CIGEVER+PIPEVER,  COCEVER, ..., METHAMEVER, PNRNMLIF+OXCNNMYR, TRQNMLIF, SEDNMLIF, STMNMLIF